# 🔧 RTL Agent Pipeline v3
**Two-Oracle: Spec-derived TB + Action-constrained repair**

```
Input (Text/PDF) → Spec IR → Writer → Verilator → Icarus (spec_tb or llm_tb)
       ↑              ↓
       └── Controller → Reviewer (FIX_PARSE|FIX_WIDTH|FIX_FUNCTION...) ←─┘
                    ↓ PASS
       Yosys (area, SVG)
```

---
**Instructions:**
1. Run Cell 1: Install iverilog, verilator, yosys, graphviz, pip
2. Run Cell 2: Set GEMINI_API_KEY in Colab Secrets
3. Run Cell 3: Load pipeline
4. Run Cell 4: Choose text or PDF → run agent

In [ ]:
# ============================================================
# CELL 1: Setup + Install Dependencies (open source, free)
# ============================================================
# If not in project folder (e.g. opened from Colab), clone repo
import os
if not os.path.exists('agents'):
    !git clone https://github.com/thotayogeswarreddy/agent.git 2>/dev/null || true
    if os.path.exists('agent'):
        %cd agent

!apt-get install -y iverilog verilator yosys graphviz -q
!pip install google-generativeai PyMuPDF Pillow -q
print('✅ iverilog, yosys, graphviz, PyMuPDF installed.')

Reading package lists...
Building dependency tree...
Reading state information...
Suggested packages:
  gtkwave
The following NEW packages will be installed:
  iverilog
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 2,130 kB of archives.
After this operation, 6,749 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 iverilog amd64 11.0-1.1 [2,130 kB]
Fetched 2,130 kB in 0s (5,644 kB/s)
Selecting previously unselected package iverilog.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../iverilog_11.0-1.1_amd64.deb ...
Unpacking iverilog (11.0-1.1) ...
Setting up iverilog (11.0-1.1) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 75.8 MB/s eta 0:00:00
✅ All dependencies installed.


In [ ]:
# CELL 2 - Replace model names with these
from google.colab import userdata
import google.generativeai as genai

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)
text_model   = genai.GenerativeModel('gemini-2.5-flash-lite')
vision_model = genai.GenerativeModel('gemini-2.5-flash-lite')
print('✅ Gemini configured.')

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


✅ Gemini configured.


In [ ]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3.1-pro-preview
models/gemini-3.1-pro-preview-customtools
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025
models/deep-research-pro-preview-12-2025


In [ ]:
# ============================================================
# CELL 3: Load Pipeline (imports from agents/, tools/)
# ============================================================
import os
from pathlib import Path

# Colab work dir
WORK_DIR = Path('/content/rtl_agent')
WORK_DIR.mkdir(exist_ok=True)
os.environ['RTL_WORK_DIR'] = str(WORK_DIR)

# Import pipeline and input layer
from input_layer import extract_from_pdf_bytes, extract_from_text
from pipeline import run_pipeline

print('✅ Pipeline loaded. Run Cell 4 (Run Agent) to start.')

✅ Pipeline code loaded. Run the next cell to start.


In [ ]:
# NEW CELL - Run this RIGHT NOW before anything else
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get('GEMINI_API_KEY'))

print("Models available to your API key:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

Models available to your API key:
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3.1-pro-preview
models/gemini-3.1-pro-preview-customtools
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025
models/deep-research-pro-preview-12-2025


In [ ]:
import google.generativeai
print(google.generativeai.__version__)

0.8.6


In [ ]:
!pip install -q --upgrade google-generativeai

In [ ]:
# ============================================================
# CELL 3b: PDF Upload + Extraction (add this before Cell 4)
# ============================================================
import fitz          # PyMuPDF
import io
import base64
from PIL import Image
from google.colab import files

def upload_and_extract_pdf() -> str:
    """
    1. Shows Colab upload button
    2. Extracts all text + images from uploaded PDF
    3. Sends everything to Gemini to distill into a hardware spec
    """
    print("📎 Click 'Choose Files' to upload your hardware spec PDF...")
    uploaded = files.upload()  # Opens Colab file picker

    if not uploaded:
        print("❌ No file uploaded.")
        return None

    # Get the uploaded file (take first if multiple)
    filename = list(uploaded.keys())[0]
    pdf_bytes = uploaded[filename]
    print(f"✅ Uploaded: {filename} ({len(pdf_bytes):,} bytes)")

    # ── Extract text + images from PDF ──────────────────────
    print("\n📄 Extracting text and images...")
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")

    all_text = ""
    all_images = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)

        # Extract text
        page_text = page.get_text()
        all_text += f"\n--- Page {page_num + 1} ---\n{page_text}"

        # Extract images
        for img in page.get_images(full=True):
            xref = img[0]
            try:
                base_img = doc.extract_image(xref)
                pil_img = Image.open(io.BytesIO(base_img["image"]))
                # Convert to RGB if needed (handles CMYK, RGBA etc.)
                if pil_img.mode not in ("RGB", "L"):
                    pil_img = pil_img.convert("RGB")
                all_images.append(pil_img)
            except Exception as e:
                print(f"  ⚠️  Skipped image on page {page_num+1}: {e}")

    print(f"  → {len(doc)} pages, {len(all_text):,} chars, {len(all_images)} images extracted")

    # ── Send everything to Gemini ────────────────────────────
    print("\n🤖 Gemini: Analyzing PDF content...")

    prompt = (
        "You are an expert hardware design engineer.\n"
        "I am giving you raw content extracted from a hardware specification document "
        "(text pages + circuit diagrams, truth tables, timing diagrams as images).\n\n"
        "Your job:\n"
        "1. Read ALL text and analyze ALL images carefully\n"
        "2. Extract every hardware-relevant detail: module names, port names, "
        "bit widths, truth tables, logic equations, timing constraints, "
        "interface specs, and any behavioral descriptions\n"
        "3. Output a clean, structured hardware specification formatted like this:\n\n"
        "MODULE NAME: <name>\n"
        "DESCRIPTION: <what it does>\n"
        "INPUTS: <list with bit widths>\n"
        "OUTPUTS: <list with bit widths>\n"
        "BEHAVIOR: <detailed logic description, truth tables, equations>\n"
        "CONSTRAINTS: <any timing or structural requirements>\n\n"
        "Be thorough — this spec will be used to generate Verilog RTL automatically.\n\n"
        f"EXTRACTED TEXT:\n{all_text}"
    )

    # Build content list: prompt text + all images
    content_parts = [prompt] + all_images

    try:
        response = vision_model.generate_content(content_parts)
        hw_spec = response.text
        print("  → Gemini distilled PDF into hardware specification ✅")
        print("\n" + "="*60)
        print("EXTRACTED HARDWARE SPECIFICATION:")
        print("="*60)
        print(hw_spec)
        print("="*60)
        return hw_spec
    except Exception as e:
        print(f"❌ Gemini extraction failed: {e}")
        print("  Falling back to raw text only...")
        return all_text.strip()


# ── Updated get_hardware_description with PDF option ────────
def get_hardware_description() -> str:
    """Choose between typing description or uploading PDF."""
    print("\n" + "="*60)
    print("RTL AGENT PIPELINE — Input")
    print("="*60)
    print("[1] Type / paste a hardware description")
    print("[2] Upload a PDF spec sheet")
    choice = input("\nChoice (1 or 2): ").strip()

    if choice == "2":
        return upload_and_extract_pdf()
    else:
        print("\nPaste your hardware description below.")
        print("Type END on a new line when done.\n")
        lines = []
        while True:
            line = input()
            if line.strip().upper() == "END":
                break
            lines.append(line)
        return "\n".join(lines)

print("✅ PDF input cell loaded. Now run Cell 4 to start.")

✅ PDF input cell loaded. Now run Cell 4 to start.


In [ ]:
# ============================================================
# CELL 4: Run the Agent (Text OR PDF)
# ============================================================
from pathlib import Path
from google.colab import files

WORK_DIR = Path('/content/rtl_agent')
WORK_DIR.mkdir(exist_ok=True)

print("="*60)
print("RTL AGENT PIPELINE v3 — Choose Input")
print("="*60)
print("[1] Type / paste hardware description")
print("[2] Upload PDF spec sheet")
choice = input("\nChoice (1 or 2): ").strip()

if choice == "2":
    print("\n📎 Click 'Choose Files' to upload your PDF...")
    uploaded = files.upload()
    if not uploaded:
        print("❌ No file uploaded.")
        spec_ir, summary = {}, ""
    else:
        filename = list(uploaded.keys())[0]
        pdf_bytes = uploaded[filename]
        print(f"✅ Uploaded: {filename}")
        print("🤖 Gemini: Extracting Spec IR from PDF...")
        spec_ir, summary = extract_from_pdf_bytes(pdf_bytes, vision_model)
        print("✅ Spec IR extracted!")
else:
    print("\nPaste description. Type END on new line when done.\n")
    lines = []
    while True:
        line = input()
        if line.strip().upper() == "END":
            break
        lines.append(line)
    raw = "\n".join(lines)
    print("🤖 Gemini: Canonicalizing to Spec IR...")
    spec_ir, summary = extract_from_text(raw, text_model)
    print("✅ Spec IR ready!")

if spec_ir and spec_ir.get("module_name"):
    print(f"\n📝 Spec: {spec_ir.get('module_name')}")
    print("-"*60)
    print(summary[:500] + ("..." if len(summary) > 500 else ""))
    print("-"*60)
    final_state = run_pipeline(spec_ir, text_model, work_dir=WORK_DIR)
else:
    print("No valid spec.")

RTL AGENT PIPELINE — Choose Input Method
[1] Type / paste a hardware description
[2] Upload a PDF spec sheet

Choice (1 or 2): 2

📎 Click 'Choose Files' to upload your PDF...


Saving Fast_Area_amp_Energy_Efficient_Supergate_Design_With_Multi-Output_amp_Multi-Functional_CDM_Cells 2.pdf to Fast_Area_amp_Energy_Efficient_Supergate_Design_With_Multi-Output_amp_Multi-Functional_CDM_Cells 2.pdf
✅ Uploaded: Fast_Area_amp_Energy_Efficient_Supergate_Design_With_Multi-Output_amp_Multi-Functional_CDM_Cells 2.pdf
  → 16 pages, 56 images extracted
🤖 Gemini analyzing PDF...
✅ Hardware spec extracted!

📝 Hardware Description (12219 chars):
------------------------------------------------------------
This document describes a Cell Design Methodology for Supergate (CDM-SG) design. The core idea is to create flexible and efficient cells that can implement multiple functions.

Here's a structured specification for Verilog RTL based on the extracted hardware-relevant details:

## CDM-SG Cell Specification

### Module Name: `cdm_sg_cell`

### Description:

This module implements a flexible supergate cell based on the Cell Design Methodology (CDM) proposed in the document. It is 

In [ ]:


# ============================================================
# CELL 5 (Optional): View saved files
# ============================================================
from pathlib import Path
WORK_DIR = Path('/content/rtl_agent')

print('=== final_dut.sv ===')
print((WORK_DIR / 'final_dut.sv').read_text() if (WORK_DIR / 'final_dut.sv').exists() else 'Not found')
print('\n=== final_tb.sv ===')
print((WORK_DIR / 'final_tb.sv').read_text() if (WORK_DIR / 'final_tb.sv').exists() else 'Not found')

=== final_dut.sv ===
Not found

=== final_tb.sv ===
Not found


In [ ]:
# ============================================================
# CELL 6 (Optional): Re-run simulation on saved files manually
# ============================================================
import subprocess
from pathlib import Path
WORK_DIR = Path('/content/rtl_agent')

dut = WORK_DIR / 'final_dut.sv'
tb  = WORK_DIR / 'final_tb.sv'
out = WORK_DIR / 'manual_sim.out'

# Compile
r = subprocess.run(
    ['iverilog', '-g2012', '-o', str(out), dut.name, tb.name],
    cwd=str(WORK_DIR), capture_output=True, text=True
)
print('Compile:', 'OK' if r.returncode == 0 else 'FAILED')
if r.stderr: print(r.stderr)

# Simulate
if r.returncode == 0:
    r2 = subprocess.run(['vvp', out.name], cwd=str(WORK_DIR), capture_output=True, text=True)
    print('\nSimulation Output:')
    print(r2.stdout)

Compile: FAILED
final_dut.sv: No such file or directory
No top level modules, and no -s option.

